In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [ ]:
def buscar_dados_processo_incremental(nup):
    # Configuração do WebDriver
    service = Service(executable_path='C:/Users/cesar/AppData/Local/Programs/Python/Python38/chromedriver.exe')
    options = Options()
    driver = webdriver.Chrome(service=service, options=options)

    try:
        # URL do site
        url = 'https://esaj.tjsp.jus.br/cpopg/open.do'
        driver.get(url)

        # Espera para garantir que a página carregue completamente
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'numeroDigitoAnoUnificado'))
        )

        # Extraí os primeiros 15 dígitos e os últimos 4 dígitos
        numero_digito_ano = nup[:15]  # '1075531-81.2021'
        foro_numero = nup[-4:]        # '0053'

        # Localiza e preenche o campo númeroDigitoAnoUnificado
        campo_processo = driver.find_element(By.ID, 'numeroDigitoAnoUnificado')
        campo_processo.send_keys(numero_digito_ano)

        # Localiza e preenche o campo foroNumeroUnificado
        campo_foro = driver.find_element(By.ID, 'foroNumeroUnificado')
        campo_foro.send_keys(foro_numero)

        # Simula a pressão da tecla Enter para submeter o formulário
        campo_foro.send_keys(Keys.RETURN)  # ou Keys.ENTER

        # Espera até que o <span> com o ID 'setasDireitamovimentacoes' esteja presente
        span_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'setasDireitamovimentacoes'))
        )

        # Rola a página para garantir que o ícone esteja visível
        icon = WebDriverWait(span_element, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'i.unj-link-collapse__icon.glyph.glyph-chevron-down'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", icon)

        # Espera até que o ícone seja clicável
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'span#setasDireitamovimentacoes i.unj-link-collapse__icon.glyph.glyph-chevron-down'))
        )

        # Usa JavaScript para clicar no ícone, se o clique tradicional não funcionar
        driver.execute_script("arguments[0].click();", icon)

        # Espera até que o <tbody> com o ID 'tabelaTodasMovimentacoes' esteja presente
        tbody = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'tabelaTodasMovimentacoes'))
        )

        # Encontra todos os <tr> dentro do <tbody>
        rows = WebDriverWait(tbody, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
        )

        # Cria listas para armazenar dados
        data = []

        # Ultima movimentação registrada # Será subistituida por uma query em um banco #
        target_date = pd.to_datetime('2024-06-13', format='%Y-%m-%d')
        target_descricao = 'Relação: 0402/2024 Data da Publicação: 14/06/2024 Número do Diário: 3986'

        # Itera sobre cada <tr> e extrai o conteúdo
        for row in rows:
            try:
                # Extrai o texto das células dataMovimentacao e descricaoMovimentacao
                data_movimentacao_text = row.find_element(By.CLASS_NAME, 'dataMovimentacao').text.strip()
                descricao_movimentacao = row.find_element(By.CLASS_NAME, 'descricaoMovimentacao').text.strip()

                # Converte a data para o formato datetime
                data_movimentacao = pd.to_datetime(data_movimentacao_text, format='%d/%m/%Y')

                # Verifica se há um link de documento # Posteriormente será subistuido pelo download do doc e upload no AWS S3 #
                documento_element = row.find_elements(By.CSS_SELECTOR, 'td[aria-hidden="true"] a.linkMovVincProc')
                documento = 'Sim' if documento_element else 'Não'

                # Divide a descrição no ponto \n
                descricao_partes = descricao_movimentacao.split('\n')

                # Monta a linha de dados
                if len(descricao_partes) >= 2:
                    descricao_principal = descricao_partes[0]
                    descricao_detalhes = '\n'.join(descricao_partes[1:])
                else:
                    descricao_principal = descricao_partes[0]
                    descricao_detalhes = ''

                row_data = [data_movimentacao, descricao_principal, descricao_detalhes, documento]

                # Adiciona dados ao dataframe apenas se a movimentação for diferente da ultima registrada
                if data_movimentacao > target_date:
                    if descricao_detalhes != target_descricao:
                        data.append(row_data)
                else:
                    # Como a data está ordenada, podemos parar de iterar se encontrarmos uma data posterior
                    break
            except Exception as e:
                print(f"Erro ao processar linha: {e}")

        # Cria um DataFrame do Pandas
        df = pd.DataFrame(data, columns=['Data Movimentação', 'Título', 'Descrição Detalhes', 'Documento'])
        df['Data Movimentação'] = pd.to_datetime(df['Data Movimentação'], format='%d/%m/%Y')

        return df
    finally:
        # Fecha o WebDriver
        driver.quit()

# Exemplo de uso
nup = '1075531-81.2021.8.26.0053'
df = buscar_dados_processo_incremental(nup)

In [ ]:
df.head()